# Introdução

O objetivo desse programa é procurar por disciplinas do catálogo da UFABC e identificar disciplinas similares.

The stop-words module was installed from: https://pypi.python.org/pypi/stop-words#installation
and using the command: pip install stop-words 


In [2]:
#from remac import remover_acentos
import string
from stop_words import get_stop_words
import csv
import numpy as np

Primeiramente obtitve o catálogo de disciplinas em formato excel, gentilmente fornecido pela Prof. Paula Tiba e sua equipe da Pró-Reitoria de Graduação. Exportei para formato csv, colocando como delimitador de campo "tab". O nome do arquivo é 

In [3]:
filename = 'catalogo2015.csv'
colSigla  = 0                     # coluna que contém as siglas das disciplinas
colEmenta = 4                     # coluna com as ementas
stopWords = ['a', 'e', 'o', 'as', 'os','ao','aos', 'da', 'de','do', 'das', 'dos', 'em', 'dos', 'para','na', 'no', 'nos', 'com', 'um','uma', 'por', 'à', 'sobre']

Para a identificação das disciplinas, compilamos todas as palavras de cada ementa e colocamos em um dicionário onde a chave é a palavra e o valor é o número de ocorrências da palavra na ementa. 

In [4]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [5]:
# removendo as palavras muito frequentes como artigos e preposições 
# as stopWords foram definidas no início da rotina
def removeStopWords(texto,stopWords):
    for sw in stopWords:                           # Laço para cada stopWord
        texto = texto.replace(' '+sw+' '," ")      # Remove as stopWords uma a uma. Foram incluídos espaços para evitar 
                                                   # remover partes das palavras
    
    return texto
    

In [6]:
def limpaTexto(texto,stopWords):
    texto = texto.translate(string.maketrans("",""), string.punctuation)
    texto = texto.lower()
    texto = removeStopWords(texto,stopWords)
    return texto

In [7]:
def criaVetor(texto):
    palavras = texto.split()                              # quebra a string em uma lista de palavras
    contagemPalavras = []                                 # inicia lista de palavras
    for w in palavras:                                   # loop para cada palavra
        contagemPalavras.append(palavras.count(w))        # conta o número de vezes que cada palavra ocorre na lista

    vetor =  dict(zip(palavras, contagemPalavras))        # Cria dicionário com as palavras e as respectivas contagens
    return vetor

In [8]:
catalogo = list(csv.reader(open(filename, 'rb'), delimiter='\t'))

Supostamente o código abaixo funcionou melhor, mas por algum motivo foi abandonado. Vou deixar aqui para uso futuro.

In [9]:
#import codecs
#with codecs.open(filename,'r',encoding='utf8') as f:
#    wordstring = f.read()

Lembro que uma das alternativas que eu havia pensado era remover os acentos. Mas acho que isso se deu por problemas em usar o utf8.

In [10]:
todasEmentas=''
len(catalogo)
for k in range(1,len(catalogo)):
    todasEmentas = todasEmentas + ' ' + catalogo[k][colEmenta]

#print todasEmentas[1:5000]     # mostrando o início das ementas


In [11]:
# removendo pontuações e stop-words
todasEmentasLimpo = limpaTexto(todasEmentas,stopWords)
print todasEmentasLimpo[1:1000]


contextualização histórica emergência teorias relações internacionais matrizes filosóficas realismo idealismo debate clássico realismo versus idealismo escola inglesa behaviorismo debate metodológico neoliberalismo neorrealismo teoria interdependência complexa neoinstitucionalismo cooperação conflito sociedade internacional anarquia regimes internacionais introdução sistemas acionamentos elétricos elementos sistema acionamento elétrico ponto operação estabilidade operação motora frenante sistema acionamento perdas acionamento elétrico operação controle máquinas corrente alternada tensão freqüência variáveis acionamentos elétricos conversores freqüência alimentados tensão algoritmos geração sinais pwm simulação acionamentos potência diodo tiristor gto transistor bipolar potência mosfet igbt acumulação energia fotossínteses fluxos energia ecossistemas características lenha características carvão vegetal acumulação combustíveis fósseis orig


## Este passo pode ser bastante demorado

Nesse ponto temos uma lista (palavras) e uma lista de quantas vezes cada palavra ocorre (contagemPalavras). Vamos agora criar um dicionário com esses pares, e ordená-lo da mais frequente para a menos frequente. 

!!!Bastante demorado!!!! 
pode levar até 5 minutos para rodar.

In [12]:
#palavras = todasEmentasLimpo.split()                  # quebra a string em uma lista de palavras
#contagemPalavras = []                                 # inicia lista de palavras
#for w in palavras:                                   # loop para cada palavra
#    contagemPalavras.append(palavras.count(w))        # conta o número de vezes que cada palavra ocorre na lista
#allPairs  =  dict(zip(palavras, contagemPalavras))    # Cria discionário com as palavras e as respectivas contagens
allPairs  = criaVetor(todasEmentasLimpo)
sortPairs = sortFreqDict(allPairs)                    # usa a função definida no início para ordenar em ordem decrescente

In [13]:
emptyPairs = {}
for aux in allPairs.keys():
    emptyPairs[aux] = 0

Por curiosidade, vamos visualizar as palavras mais frequentes

In [14]:
for k in range(20):
    # just for visualization, let's see the mostr frequent words...
    print str(k+1) + ": " + str(sortPairs[k][1]) + ' ==> '+ str(sortPairs[k][0]) + ' vezes'
    

1: sistemas ==> 383 vezes
2: análise ==> 260 vezes
3: desenvolvimento ==> 204 vezes
4: introdução ==> 193 vezes
5: teoria ==> 175 vezes
6: processos ==> 173 vezes
7: técnicas ==> 170 vezes
8: energia ==> 168 vezes
9: políticas ==> 165 vezes
10: modelos ==> 165 vezes
11: trabalho ==> 163 vezes
12: conceitos ==> 163 vezes
13: aplicações ==> 155 vezes
14: métodos ==> 142 vezes
15: materiais ==> 141 vezes
16: projeto ==> 135 vezes
17: curso ==> 134 vezes
18: principais ==> 132 vezes
19: sistema ==> 128 vezes
20: planejamento ==> 127 vezes


In [15]:
#Abandonei essa ideia, a de apagar as palavras mais frequentes, mas vou deixar aqui caso volte a usar...
#NRemove = int(10)
#for k in range(NRemove):
#    del allPairs[sortPairs[k][1]]
#    
#print "Pairs\n" + str(allPairs)

Gerando vetores para palavras frequentes. Tirado de http://stackoverflow.com/questions/10648490/removing-first-appearance-of-word-from-a-string

In [16]:
V = np.zeros((len(catalogo), len(emptyPairs)),dtype=int)

#print len(catalogo)

l = len(emptyPairs)
#print l

for k in range(1,len(catalogo)):
    estaSigla  = catalogo[k][colSigla ] 
    estaEmenta = catalogo[k][colEmenta] 
    
    
    estaEmentaLimpa = limpaTexto(estaEmenta,stopWords)
    esteVetor = criaVetor(estaEmentaLimpa)
    
    vetorCompleto = emptyPairs.copy()
    vetorCompleto.update(esteVetor)
    
    if len(vetorCompleto) != l:
        #print estaEmentaLimpa
    
        #print k, len(vetorCompleto) 
        
        s1 = set(vetorCompleto.keys())
        s2 = set(allPairs.keys())
        s1.difference_update(s2)
        
        #print s1
        for aux in s1:
            del vetorCompleto[aux]
                
    V[k][:] = np.array(vetorCompleto.values())
    #V = np.array(vetorCompleto.values())
        
print V
print np.sum(V,1)
print '\n'

#M = np.matrixmultiply(V,V.transpose())
M = np.inner(V,V)

print M


[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[ 0 33 48 ..., 25 22 29]


[[ 0  0  0 ...,  0  0  0]
 [ 0 41  0 ...,  0  0  0]
 [ 0  0 78 ...,  0  0  0]
 ..., 
 [ 0  0  0 ..., 31  5  6]
 [ 0  0  0 ...,  5 24  4]
 [ 0  0  0 ...,  6  4 31]]


In [81]:
(I,J) = (M>40).nonzero()
I.size

2299

In [99]:
s='Trabalho de Graduação I em Engenharia Aeroespacial'
s=s.lower().split()
p='graduação'
p in set(s) and 'a' not in set(s)

True

In [108]:
#print I,J,M[I,J]
coef = int;
for k in range(I.size):
    coef = float(M[I[k],J[k]]*M[I[k],J[k]])/float(M[I[k],I[k]]*M[J[k],J[k]])
    ement = catalogo[I[k]][1].lower().split()
    if 0.3 < coef < 2 and I[k] < J[k] and 'graduação' not in set(ement) and 'estágio' not in set(ement) and 'trabalho' not in set(ement) and 'tcc' not in set(ement):
        print 'M[ij]=',M[I[k],J[k]],' M[ii]=',M[I[k],I[k]], 'M[jj]=',M[J[k],J[k]], coef
        print catalogo[I[k]][1]
        print catalogo[I[k]][colEmenta]
        print '\n'
        print catalogo[J[k]][1]
        print catalogo[J[k]][colEmenta]
        print '\n\n'              

M[ij]= 102  M[ii]= 118 M[jj]= 271 0.325348677216
Análise Real I
Números reais: propriedades e completeza. Seqüências e séries de números reais. Topologia da Reta: conjuntos abertos e fechados, pontos de acumulação, conjuntos compactos e conjunto de cantor. Limite de funções reais. Funções contínuas: definição, funções contínuas num intervalo, funções contínuas em conjuntos compactos e continuidade uniforme. Funções deriváveis: definição de derivada, derivada e crescimento local, funções deriváveis num intervalo, fórmula de Taylor, aplicações da derivada, concavidade e convexidade.


Bases Matemáticas
Elementos de Linguagem e Lógica Matemática: proposições, conectivos e quantificadores, condições necessária e suficiente. Elementos da Teoria Ingênua de Conjuntos: Conjuntos, Subconjuntos, Operações com Conjuntos: União e Intersecção. Conjuntos Numéricos: Números naturais e Indução. Números Reais. Equações e Inequações. Funções: definição e propriedades. Funções Injetoras e Sobrejetoras. O